In [13]:
import sys
sys.path.append("../../")

In [28]:
from multi_tp.models_ids import *
from multi_tp.utils import get_model_name_path, get_suffix, LANGUAGES, performance_file_v2_tmpl, pivot_file_tmpl, cache_parse_responses_tmpl, cache_responses_tmpl
import os
system_role = "normal"
translator_provider_forward = "google"
translator_provider_backward = "google"
analysis_backend_model_version = "meta-llama/Meta-Llama-3.1-8B-Instruct"
add_paraphrase = False
country = None

for model_version in MODELS:
    for lang in LANGUAGES:
        in_path = cache_parse_responses_tmpl.format(
                    model_version=get_model_name_path(model_version),
                    system_role=system_role,
                    lang=lang,
                    suffix=get_suffix(add_paraphrase, country),
                    translator_provider_forward=translator_provider_forward,
                    translator_provider_backward=translator_provider_backward,
                    analysis_backend_model_version=get_model_name_path(
                        analysis_backend_model_version
                    ),
                )
        import pandas as pd
        BASE_DIR ="../../"
        in_path = os.path.join(BASE_DIR, in_path)
        if not os.path.exists(in_path):
            continue
        df = pd.read_csv(in_path)

        data_source = df["gpt_response_en"]

        query=f"Does the following answer make sense? {data_source}"

KeyboardInterrupt: 

In [27]:
cache_parse_responses_tmpl

'data/cache_parsing/B={analysis_backend_model_version}/{model_version}_{system_role}_{lang}+{translator_provider_forward}{suffix}_tr+{translator_provider_backward}_response.csv'

In [14]:
from multi_tp.models_ids import *
from multi_tp.utils import get_model_name_path, get_suffix, LANGUAGES, performance_file_v2_tmpl, pivot_file_tmpl, cache_parse_responses_tmpl, cache_responses_tmpl
import os
system_role = "normal"
translator_provider_forward = "google"
translator_provider_backward = "google"
analysis_backend_model_version = "meta-llama/Meta-Llama-3.1-8B-Instruct"
add_paraphrase = False
country = None


model_version=GPT_4_OMNI_MINI

acc = []
for model_version in MODELS:
    for lang in LANGUAGES:
        in_path = cache_parse_responses_tmpl.format(
                    model_version=get_model_name_path(model_version),
                    system_role=system_role,
                    lang=lang,
                    suffix=get_suffix(add_paraphrase, country),
                    translator_provider_forward=translator_provider_forward,
                    translator_provider_backward=translator_provider_backward,
                    analysis_backend_model_version=get_model_name_path(
                        analysis_backend_model_version
                    ),
                )
        import pandas as pd
        BASE_DIR ="../../"
        in_path = os.path.join(BASE_DIR, in_path)
        if not os.path.exists(in_path):
            continue
        df = pd.read_csv(in_path)
        # Apply the analysis to your DataFrame
        def get_category(row):
            prob = row['this_saving_prob']
            if prob == 0.5:  # either case
                return 'either'
            elif prob == -1:  # neither case
                return 'neither'
            elif prob == -10:  # underskilled case
                return 'underskilled'
            return 'valid'  # for any other cases
        
        # Add category column
        df['category'] = df.apply(get_category, axis=1)
        # result_df['model_version'] = model_version
        # result_df['lang'] = lang
        tmp = df["category"].value_counts() / len(df) * 100
        tmp = pd.DataFrame(tmp).T
        tmp['model_version'] = model_version
        tmp['lang'] = lang
        tmp["legible"] = 100 - tmp["underskilled"] if "underskilled" in tmp else 100
        acc.append(tmp)

result_df = pd.concat(acc)


In [20]:
#  LLAMA_2_70B, LLAMA_3_1_70B
acc = []
key = "valid"
for model in MODELS:
    a = result_df[result_df["model_version"].isin([model])]
    if len(a) == 0:
        continue
    acc.append(pd.DataFrame({
        "model_version": get_model_name_path(model),
        "Perc. language (valid >= 80) ":len(a[a[key] >= 80]) / len(a) * 100,
        "Perc. language (80 > valid >= 70)":len(a[a[key].apply(lambda x : x >= 70 and x < 80)]) / len(a) * 100,
        "Perc. language (valid < 70)":len(a[a[key].apply(lambda x : x < 70)]) / len(a) * 100,
    }, index=[0]))
print(pd.concat(acc).to_markdown(index=False))

| model_version                               |   Perc. language (valid >= 80)  |   Perc. language (80 > valid >= 70) |   Perc. language (valid < 70) |
|:--------------------------------------------|--------------------------------:|------------------------------------:|------------------------------:|
| mistralai_Mistral-7B-Instruct-v0.2          |                       11.215    |                             8.41121 |                       80.3738 |
| meta-llama_Meta-Llama-3.1-8B-Instruct       |                        5.60748  |                            14.9533  |                       79.4393 |
| neuralmagic_Meta-Llama-3.1-70B-Instruct-FP8 |                       28.3019   |                            28.3019  |                       43.3962 |
| meta-llama_Meta-Llama-3-8B-Instruct         |                       39.2157   |                            20.5882  |                       40.1961 |
| neuralmagic_Meta-Llama-3-70B-Instruct-FP8   |                       50.4673   |       

In [22]:
#  LLAMA_2_70B, LLAMA_3_1_70B
acc = []
key = "legible"
for model in MODELS:
    a = result_df[result_df["model_version"].isin([model])]
    if len(a) == 0:
        continue
    acc.append(pd.DataFrame({
        "model_version": get_model_name_path(model),
        "skilled_95":len(a[a[key] >= 95]) / len(a) * 100,
        "skilled_90":len(a[a[key].apply(lambda x : x > 90 and x <= 95)]) / len(a) * 100,
        "skilled_below_90":len(a[a[key].apply(lambda x : x < 90)]) / len(a) * 100,
    }, index=[0]))
print(pd.concat(acc).to_markdown(index=False))


| model_version                               |   skilled_95 |   skilled_90 |   skilled_below_90 |
|:--------------------------------------------|-------------:|-------------:|-------------------:|
| mistralai_Mistral-7B-Instruct-v0.2          |     100      |     0        |                  0 |
| meta-llama_Meta-Llama-3.1-8B-Instruct       |     100      |     0        |                  0 |
| neuralmagic_Meta-Llama-3.1-70B-Instruct-FP8 |     100      |     0        |                  0 |
| meta-llama_Meta-Llama-3-8B-Instruct         |     100      |     0        |                  0 |
| neuralmagic_Meta-Llama-3-70B-Instruct-FP8   |     100      |     0        |                  0 |
| meta-llama_Llama-2-70b-chat-hf              |     100      |     0        |                  0 |
| meta-llama_Llama-2-13b-chat-hf              |      98.1308 |     1.86916  |                  0 |
| meta-llama_Llama-2-7b-chat-hf               |      99.0654 |     0.934579 |                  0 |
| Qwen_Qwe